In [1]:
import numpy as np
from neural_networks import NeuralNet
import scipy.special as sp

In [2]:
#Data load
#X_train = np.load('/Users/yasminebenmessaoud/Library/CloudStorage/OneDrive-ITU/Machine Learning/JustTwo/post_pca_data/X_train.npy')
#Y_train = np.load('/Users/yasminebenmessaoud/Library/CloudStorage/OneDrive-ITU/Machine Learning/JustTwo/post_pca_data/Y_train.npy')


In [3]:
X_train = np.load('..\\..\\post_pca_data\\X_train_initial.npy')
X_validate = np.load('..\\..\\post_pca_data\\X_validate_initial.npy')
Y_train = np.load('..\\..\\post_pca_data\\Y_train_initial.npy')
Y_validate = np.load('..\\..\\post_pca_data\\Y_validate_initial.npy')

In [4]:
def ReLU(x):
    return x * (x > 0)

#tried using this as the activation in hidden layers, and model got insignificantly worse (5 more incorrect)

In [5]:
class NeuralNet(object):
    RNG = np.random.default_rng()

    def __init__(self, topology: list[int] = []):
        self.topology = topology
        self.weight_mats = []

        self._init_matrices()

    def _init_matrices(self):
        # Set up weight matrices
        if len(self.topology) > 1:
            j = 1
            for i in range(len(self.topology) - 1):
                num_rows = self.topology[i]
                num_cols = self.topology[j]

                mat = self.RNG.random(size=(num_rows, num_cols))
                self.weight_mats.append(mat)

                j += 1

    def feedforward(self, input_vector):
        I = input_vector

        for idx, W in enumerate(self.weight_mats):
            I = np.dot(I, W)

            if idx == len(self.weight_mats) - 1:
                out_vector = sp.softmax(I)  # Output layer
            else:
                I = sp.softmax(I)  # Hidden layers

        return out_vector


In [6]:
#Define topology based on data's feature size and desired layers
input_size = X_train.shape[1]  # Number of features in your data
nnet = NeuralNet(topology=[input_size, 4, 5])  # Example topology: input -> 4 hidden -> 2 output

In [7]:
# Perform feedforward on the first sample
output = nnet.feedforward(X_train[500])  # Pass the first data point to the network
print(output)

[0.19161312 0.14907715 0.19715729 0.15914187 0.30301058]


In [8]:
#Train the model
def cross_entropy_loss(predictions, true_labels):
    return -np.sum(true_labels * np.log(predictions + 1e-9)) / true_labels.shape[0]


In [9]:
def train(neural_net, X_train, y_train, learning_rate, epochs):
    for epoch in range(epochs):
        epoch_loss = 0
        for x, y_true in zip(X_train, y_train):
            # Forward pass
            y_pred = neural_net.feedforward(x)
            
            # Compute loss
            loss = cross_entropy_loss(y_pred, y_true)
            epoch_loss += loss
            
            # Backpropagation (compute gradients) - this part needs to be implemented
            
            # Update weights - modify neural_net.weight_mats using the gradients
        
        print(f"Epoch {epoch + 1}/{epochs}, Loss: {epoch_loss / len(X_train)}")


In [10]:
def train(neural_net, X_train, y_train, learning_rate, epochs):
    def softmax_derivative(output):
        # Softmax derivative for backpropagation
        return output * (1 - output)
    
    for epoch in range(epochs):
        epoch_loss = 0
        
        for x, y_true in zip(X_train, y_train):
            # Forward pass
            activations = [x]  # Store activations for backpropagation
            current_input = x

            # Forward propagate through layers
            for W in neural_net.weight_mats:
                current_input = sp.softmax(np.dot(current_input, W))  # Activation
                activations.append(current_input)

            y_pred = activations[-1]

            # Compute loss
            loss = cross_entropy_loss(y_pred, y_true)
            epoch_loss += loss

            # Backpropagation
            deltas = []  # Store deltas for each layer
            delta = y_pred - y_true  # Output layer delta
            deltas.append(delta)

            # Compute gradients for each layer (backpropagate)
            for i in range(len(neural_net.weight_mats) - 1, 0, -1):
                delta = (deltas[-1] @ neural_net.weight_mats[i].T) * softmax_derivative(activations[i])
                deltas.append(delta)

            deltas.reverse()  # Reverse deltas to match layers

            # Update weights
            for i in range(len(neural_net.weight_mats)):
                grad = np.outer(activations[i], deltas[i])  # Gradient of the weights
                neural_net.weight_mats[i] -= learning_rate * grad  # Gradient descent step
        
        # Print epoch loss
        print(f"Epoch {epoch + 1}/{epochs}, Loss: {epoch_loss / len(X_train)}")


In [11]:
from sklearn.preprocessing import OneHotEncoder

encoder = OneHotEncoder(sparse_output=False)
y_train_one_hot = encoder.fit_transform(Y_train.reshape(-1, 1))


In [12]:
# Define training parameters
learning_rate = 0.01
epochs = 100

# Train the model
train(nnet, X_train, y_train_one_hot, learning_rate, epochs)


Epoch 1/100, Loss: 0.3216487703374046
Epoch 2/100, Loss: 0.32234287216970126
Epoch 3/100, Loss: 0.32234287202193257
Epoch 4/100, Loss: 0.32234287202193257
Epoch 5/100, Loss: 0.32234287202193257
Epoch 6/100, Loss: 0.32234287202193257
Epoch 7/100, Loss: 0.32234287202193257
Epoch 8/100, Loss: 0.32234287202193257
Epoch 9/100, Loss: 0.32234287202193257
Epoch 10/100, Loss: 0.32234287202193257
Epoch 11/100, Loss: 0.32234287202193257
Epoch 12/100, Loss: 0.32234287202193257
Epoch 13/100, Loss: 0.32234287202193257
Epoch 14/100, Loss: 0.32234287202193257
Epoch 15/100, Loss: 0.32234287202193257
Epoch 16/100, Loss: 0.32234287202193257
Epoch 17/100, Loss: 0.32234287202193257
Epoch 18/100, Loss: 0.32234287202193257
Epoch 19/100, Loss: 0.32234287202193257
Epoch 20/100, Loss: 0.32234287202193257
Epoch 21/100, Loss: 0.32234287202193257
Epoch 22/100, Loss: 0.32234287202193257
Epoch 23/100, Loss: 0.32234287202193257
Epoch 24/100, Loss: 0.32234287202193257
Epoch 25/100, Loss: 0.32234287202193257
Epoch 26/1

In [13]:
# Predict for a new data sample (e.g., the first row of your dataset)
new_sample = X_train[0]
prediction = nnet.feedforward(new_sample)

# Convert the output (softmax probabilities) to a predicted class
predicted_class = np.argmax(prediction)
print(f"Predicted Class: {predicted_class}")


Predicted Class: 4


In [14]:
import numpy as np
import scipy.special as sp
from sklearn.preprocessing import OneHotEncoder


# Neural Network Class
class NeuralNet:
    RNG = np.random.default_rng()

    def __init__(self, topology: list[int]):
        self.topology = topology
        self.weight_mats = []
        self._init_matrices()

    def _init_matrices(self):
        for i in range(len(self.topology) - 1):
            num_rows = self.topology[i]
            num_cols = self.topology[i + 1]
            mat = self.RNG.random(size=(num_rows, num_cols))
            self.weight_mats.append(mat)

    def feedforward(self, input_vector):
        I = input_vector
        for idx, W in enumerate(self.weight_mats):
            I = np.dot(I, W)
            I = sp.softmax(I)
        return I


# Cross-Entropy Loss Function
def cross_entropy_loss(predictions, true_labels):
    return -np.sum(true_labels * np.log(predictions + 1e-9)) / predictions.shape[0]


# Training Function
def train(neural_net, X_train, y_train, learning_rate, epochs):
    def softmax_derivative(output):
        return output * (1 - output)

    for epoch in range(epochs):
        epoch_loss = 0
        for x, y_true in zip(X_train, y_train):
            # Forward pass
            activations = [x]
            current_input = x
            for W in neural_net.weight_mats:
                current_input = sp.softmax(np.dot(current_input, W))
                activations.append(current_input)
            y_pred = activations[-1]

            # Compute loss
            loss = cross_entropy_loss(y_pred, y_true)
            epoch_loss += loss

            # Backpropagation
            deltas = [y_pred - y_true]  # Output layer delta
            for i in range(len(neural_net.weight_mats) - 1, 0, -1):
                delta = (deltas[-1] @ neural_net.weight_mats[i].T) * softmax_derivative(activations[i])
                deltas.append(delta)
            deltas.reverse()

            # Update weights
            for i in range(len(neural_net.weight_mats)):
                grad = np.outer(activations[i], deltas[i])
                neural_net.weight_mats[i] -= learning_rate * grad

        print(f"Epoch {epoch + 1}/{epochs}, Loss: {epoch_loss / len(X_train)}")


# Main Code
if __name__ == "__main__":
    # Load data
    X_train = np.load('..\\..\\post_pca_data\\X_train_initial.npy')
    X_validate = np.load('..\\..\\post_pca_data\\X_validate_initial.npy')
    Y_train = np.load('..\\..\\post_pca_data\\Y_train_initial.npy')
    Y_validate = np.load('..\\..\\post_pca_data\\Y_validate_initial.npy')

    # One-hot encode labels
    encoder = OneHotEncoder(sparse_output=False)
    y_train_one_hot = encoder.fit_transform(Y_train.reshape(-1, 1))

    # Define topology and initialize the neural network
    input_size = X_train.shape[1]
    output_size = y_train_one_hot.shape[1]
    nnet = NeuralNet(topology=[input_size, 4, output_size])

    # Train the network
    learning_rate = 0.01
    epochs = 100
    train(nnet, X_train, y_train_one_hot, learning_rate, epochs)

    # Predict for a new data sample
    new_sample = X_train[0]
    prediction = nnet.feedforward(new_sample)
    predicted_class = np.argmax(prediction)
    print(f"Predicted Class: {predicted_class}")


Epoch 1/100, Loss: 0.32258114539473903
Epoch 2/100, Loss: 0.32209372076637915
Epoch 3/100, Loss: 0.32234266377109116
Epoch 4/100, Loss: 0.32234234124340616
Epoch 5/100, Loss: 0.3223421438592987
Epoch 6/100, Loss: 0.32234194728500426
Epoch 7/100, Loss: 0.3223417515218698
Epoch 8/100, Loss: 0.3223415565712005
Epoch 9/100, Loss: 0.32234136243424844
Epoch 10/100, Loss: 0.32234116911221206
Epoch 11/100, Loss: 0.3223409766062362
Epoch 12/100, Loss: 0.3223407849174117
Epoch 13/100, Loss: 0.32234059404677523
Epoch 14/100, Loss: 0.32234040399530933
Epoch 15/100, Loss: 0.322340214763942
Epoch 16/100, Loss: 0.32234002635354647
Epoch 17/100, Loss: 0.32233983876494154
Epoch 18/100, Loss: 0.32233965199889103
Epoch 19/100, Loss: 0.32233946605610375
Epoch 20/100, Loss: 0.32233928093723363
Epoch 21/100, Loss: 0.32233909664287946
Epoch 22/100, Loss: 0.3223389131735849
Epoch 23/100, Loss: 0.3223387305298384
Epoch 24/100, Loss: 0.32233854871207346
Epoch 25/100, Loss: 0.32233836772066815
Epoch 26/100, Loss

In [20]:
# Train the network
learning_rate = 0.01
epochs = 100
train(nnet, X_train, y_train_one_hot, learning_rate, epochs)

# Predict for a new data sample
new_sample = X_train[56]
prediction = nnet.feedforward(new_sample)
predicted_class = np.argmax(prediction)
print(f"Predicted Class: {predicted_class}")

Epoch 1/100, Loss: 0.32236830157558916
Epoch 2/100, Loss: 0.3223074503760843
Epoch 3/100, Loss: 0.3223074412415385
Epoch 4/100, Loss: 0.3223074321407
Epoch 5/100, Loss: 0.32230742307202176
Epoch 6/100, Loss: 0.3223074140353452
Epoch 7/100, Loss: 0.32230740503051253
Epoch 8/100, Loss: 0.3223073960573671
Epoch 9/100, Loss: 0.32230738711575324
Epoch 10/100, Loss: 0.32230737820551614
Epoch 11/100, Loss: 0.32230736932650206
Epoch 12/100, Loss: 0.32230736047855807
Epoch 13/100, Loss: 0.3223073516615323
Epoch 14/100, Loss: 0.3223073428752737
Epoch 15/100, Loss: 0.3223073341196322
Epoch 16/100, Loss: 0.3223073253944587
Epoch 17/100, Loss: 0.322307316699605
Epoch 18/100, Loss: 0.3223073080349237
Epoch 19/100, Loss: 0.32230729940026825
Epoch 20/100, Loss: 0.3223072907954933
Epoch 21/100, Loss: 0.3223072822204541
Epoch 22/100, Loss: 0.32230727367500683
Epoch 23/100, Loss: 0.3223072651590087
Epoch 24/100, Loss: 0.3223072566723175
Epoch 25/100, Loss: 0.3223072482147922
Epoch 26/100, Loss: 0.3223072

In [21]:
#for saving/loading the model
import pickle

In [22]:
#save the model
filename = 'our_FFNN_model.sav'

pickle.dump(nnet, open(filename, 'wb'))

In [23]:
#load the model
saved_model = pickle.load(open(filename, 'rb'))


# Test the model

In [24]:
# Predictions for X_validate

validation_predictions = []

for datapoint in X_validate:
    prediction = nnet.feedforward(datapoint)
    predicted_class = np.argmax(prediction)
    validation_predictions.append(predicted_class)

print('Correct prediction count')
print(sum(validation_predictions == Y_validate))

print('Correct prediction ratio')
print(sum(validation_predictions == Y_validate)/len(Y_validate))


Correct prediction count
403
Correct prediction ratio
0.2015
